# Agentes con Transformers

*Este notebook es una adaptación de:* https://huggingface.co/docs/transformers/main/en/agents

Los modelos de lenguaje tienen una indudable capacidad para intercambiar información a través del lenguaje natural. Sin embargo, a veces muestran una sorprendente incapacidad para realizar tareas sencillas, especialmente en ciertos dominios como el razonamiento lógico, los cálculos matemáticos o la búsqueda de información.

Una aproximación para superar esta limitación es el uso de *agentes*. Un agente es un sistema basado en el uso de un modelo de lenguaje como motor, y que además está equipado con acceso a ciertas herramientas (*tools*) que puede emplear para realizar tareas muy específicas. Por ejemplo, un agente podría estar preparado para que cuando interprete que alguien le pide que realice un cálculo matemático, utilizar algún tipo de software matemático para realizarlo.

Los agente pueden ser programados para:

- Concebir una serie de acciones/herramientas y ejecutarlas todas a la vez, como el [CodeAgent](https://huggingface.co/docs/transformers/main/en/main_classes/agent#transformers.CodeAgent)
- Planear y luego ejecutar una serie de acciones (una a una) esperando a que cada una se complete antes de continua con la siguiente, como el [ReactJsonAgent](https://huggingface.co/docs/transformers/main/en/main_classes/agent#transformers.ReactJsonAgent)

## Tipos de agentes

### Agentes Code

Este agente tiene un paso de planificación que genera código Python para ejecutar todas sus acciones de una vez. Nativamente es capaz de manejar diferentes tipos de entradas y salidas, por lo tanto es la opción recomendada para tareas multimodales.

### Agentes React

Este es el agente recomendado para resolver tareas de razonamiento, ya que el framework React es eficiente para pensar paso a paso, y cada paso usando las salidas del paso anterior. Transformers implementa dos versiones del *ReactJsonAgent*:

- *ReactJsonAgent*: genera llamadas a herramientas en forma de JSON en su salida.
- *ReactCodeAgente*: e un nuevo tipo de *ReactJsonAgent* que genera sus llamadas a herramientas como *blobs* (binarios ejectuables) de código.


## Construcción de un agente

La inicialización de un agente requiere los siguientes argumentos:

-  Un modelo de lenguaje para equipar a tu agente.
-  Un *prompt* inicial para inicializar el modelo de lenguaje.
-  Una caja de herramientas de la que el agente puede coger diferente herramientas (*tools*).
-  Un *parser* para extraer del modelo de lenguaje qué herramienta se van a llamar y con qué argumentos.

Al inicializar el sistema del agente, los atributos de las herramientas se utilizan para generar una descripción de las mismas, que luego se integran en el system_prompt del agente para informarle qué herramientas puede usar y por qué.

En primer lugar, la funcionalidad de agentes se proporciona como un extra que se debe instalar de forma separada. Además, debemos actualizar la versión de transformers a la última disponible, que no está disponible como paquete pip por lo que debemos cogerla y construirla desde el repositorio git.

En primer lugar vamos a construir tu *engine* de modelo de lenguaje que acepta una lista de mensaje y devuelve un texto generado. Este invocable también acepta un argumento *stop* que indica cuándo parar de generar.

Para ello, vamos a utilizar un API token de HuggingFace, por lo tanto, regístrate en HuggingFace y obtiene tu API token para rellena la clave en la siguiente celda. En su capa gratuita, se permiten 1000 peticiones al API de inferencia cada día.

En https://huggingface.co/settings/tokens :
- Regístrate si no lo has hecho anteriormente.
- Crea un nuevo token
- Elige la opción (*tab*) *fine-grained*
- Marca las tres casilla de *Inference*
- Pulsa el botón "Create new token"

En la siguiente página copia el access token a la siguiente casilla sustityendo a <YOUR_HUGGINGFACEHUB_API_TOKEN>

La lista de endpoints compatibles (en la categoría de *text generation*) disponibles se puede consultar aquí: https://huggingface.co/models?inference=warm&pipeline_tag=text-generation&other=endpoints_compatible&sort=trending


In [18]:
from huggingface_hub import login, InferenceClient

login("<YOUR_HUGGINGFACEHUB_API_TOKEN>")

client = InferenceClient(model="microsoft/Phi-3.5-mini-instruct")

def llm_engine(messages, stop_sequences=["Task"]) -> str:
    response = client.chat_completion(messages, stop=stop_sequences, max_tokens=1000)
    answer = response.choices[0].message.content
    return answer

Se puede usar cualquier *llm_engine* siempre que:

1. Sigue los formatos de mensa (List[Dict[str, str]]) para sus mensaje de entrada y devuelve la respuesta como *str*.
2. Para de generar salidas al recibir las secuencias pasadas como  argumento stop_sequences. 

Además, *llm_engine* también puede recibir un argumento *grammar*. Este argumento a su vez será enviado a las llamadas al *llm_engine* junto con el *grammar* usado para la inicialización del agente. Esto permite la generación restringida (*constrained generation*) para forzar salidas del agente bien formadas.

También es necesario un argumento *tools* que debe contener una lista de herramientas (*Tools*), el cual puede inicializarse vacío. También se puede añadir la caja de herramientas (*toolbox*) por defecto sobre tu lista de herramientas, definiendo el argumento opcional *add_base_tools=True*.

A continuación, vamos a crear un agente de tipo *CodeAgent* y vamos a ejecutarlo. También es posible crear un *TransformerEngine* con un pipeline preinicializado para ejecutar la inferencia en tu máquina local usando *transformers*. Por comodidad, como los comportamientos del agente requiere modelos de lenguaje pontentes como Llama-3.1-70B-Instruct que son difíciles de correr localmente, también proporcionamos la clase HfApiEngine que inicializa el huggingface_hub.InferenceClient por debajo.

In [19]:
from transformers import HfApiEngine, CodeAgent

llm_engine = HfApiEngine(model="microsoft/Phi-3.5-mini-instruct")
agent = CodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)

agent.run(
    "Could you translate this sentence from French to English?",
    sentence="Où est la boulangerie la plus proche?",
)

======== New task ========
Could you translate this sentence from French to English?
You have been provided with these initial arguments: {'sentence': 'Où est la boulangerie la plus proche?'}.
=== Agent thoughts:

>>> Agent is executing the code below:
translated_sentence = translator(text="Où est la boulangerie la plus proche?", src_lang="French", tgt_lang="English")
print(f"The translated sentence is {translated_sentence}.")
====
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Incluso si no hubiéramos creado un *HfApiEngine* para poblar el argumento *llm_engine* del *CodeAgent*, se habría creado un *engine* por defecto.

Fíjate además que el argumento *sentence* se ha usado para pasasr información adicional al modelo. También podemos usar argumentos como estos para pasar rutas a ficheros locales o remotos.

In [20]:
from transformers import ReactCodeAgent

agent = ReactCodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)

#agent.run("Why does Mike not know many people in New York?", audio="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/recording.mp3")

Los campos *prompt* y *output parser* han sido definidos automáticamente, pero se puede ver con qué valores se han inicializado en el campo *system_prompt_template* de tu agente.

In [21]:
print(agent.system_prompt_template)


You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_action>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using

Es importante explicar tan claramente como seal posible la tareas que quieres realizar. Cada ejecución de *run()* es independiente, y ya que un agente está equipado con un LLM, variaciones menores en tu *prompt* podrían producir resultados completamente diferentes. También puedes ejecutar un agente consecutivamente para diferentes tareas: en cada ejecución, los atributos *agent.task* y *agent.logs* serán reinicializados.

### Ejecución de código

Un intérprete de Python ejecuta el código sobre un conjunto de entradas que se pasan junto con tus herramientas. Esto debería ser seguro porque las única funciones que se pueden invocar se corresponden con las herramientas que proporcionaste (especialmente si nos restringimos a las herramientas proporcionadas por HuggingFace) y a la función print.

Además, el intérprete de Python no permite *imports* fuera de la lista segurda, por lo tanto los ataques más obvios no deberían ser un problema. Aún así puedes autorizar *imports* adicionales pasando una lista de módulos autorizados como una lista de strings al argumento *additional_authorized_imports* cuando se inicializa el agente.

In [22]:
from transformers import ReactCodeAgent
llm_engine = HfApiEngine(model="microsoft/Phi-3.5-mini-instruct")

agent = ReactCodeAgent(tools=[], llm_engine=llm_engine, additional_authorized_imports=['requests', 'bs4'])
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")


======== New task ========
Could you get me the title of the page at url 'https://huggingface.co/blog'?
=== Agent thoughts:
Thought: I need to fetch the title of the page from the provided URL. I will use the `requests` module to send an HTTP GET request to the URL and then extract the title from the HTML content using the `BeautifulSoup` module.
>>> Agent is executing the code below:
import requests
from bs4 import BeautifulSoup

url = 'https://huggingface.co/blog'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

page_title = soup.title.string
print("Page title:", page_title)
====
Print outputs:
Page title: Hugging Face – Blog

=== Agent thoughts:
Thought: The print output has provided the title of the page from the URL. Now I will use the `final_answer` tool to present the final answer in the required format.
>>> Agent is executing the code below:
final_answer({'answer': {'type': 'string', 'description': 'The title of the page at https://huggingface

{'answer': {'type': 'string',
  'description': 'The title of the page at https://huggingface.co/blog',
  'value': 'Hugging Face – Blog'}}

La ejecución parará si cualquier código intenta realizar una operación ilegal, o si hay un error de Python convencional generado por el código del agente.

Como el modelo de lenguaje puede generar código arbitrario para ser ejecutado, evita añadir *imports* inseguros.

### El *prompt* de sistema

Un agente, o el modelo de lenguaje que guía al agente, genera salidas basadas en un *prompt* de sistema. Este *prompt* puede ser diseñado para la tarea que se quiere realizar. Por ejemplo, aquí tienes el *prompt* de sistema por defecto para el *ReactCodeAgent*

### El *prompt* de sistema

Un agente, o el modelo de lenguaje que guía al agente, genera salidas basadas en un *prompt* de sistema. Este *prompt* puede ser diseñado para la tarea que se quiere realizar. Por ejemplo, aquí tienes el *prompt* de sistema por defecto para el *ReactCodeAgent*

> You will be given a task to solve as best you can.
> You have access to the following tools:
> <<tool_descriptions>>
>
> To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.
>
> At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task, then the tools that you want to use.
> Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '/End code' sequence.
> During each intermediate step, you can use 'print()' to save whatever important information you will then need.
> These print outputs will then be available in the 'Observation:' field, for using this information as input for the next step.
>
> In the end you have to return a final answer using the `final_answer` tool.
>
> Here are a few examples using notional tools:
> ---
> {examples}
>
> Above example were using notional tools that might not exist for you. You only have acces to those tools:
> <<tool_names>>
> You also can perform computations in the python code you generate.
>
> Always provide a 'Thought:' and a 'Code:\n```py' sequence ending with '```<end_code>' sequence. You MUST provide at least the 'Code:' sequence to move forward.
>
> Remember to not perform too many operations in a single code block! You should split the task into intermediate code blocks.
> Print results at the end of each step to save the intermediate results. Then use final_answer() to return the final result.
>
> Remember to make sure that variables you use are all defined.
>
> Now Begin!*



El prompt de sistema incluye:

- Una *introducción* que explica cómo se debe comportar el agente y qué herramientas están disponibles.
- Una *descripción* de todas las herramientas disponibles que es definida por un token <<tools_descriptions>> que es dinámicamente reemplazado en tiempo de ejecución con las herramientas definidas/elegidas por el usuario. Esta descipción proviene de los atributos de las herramientas (*name*,*description*,*inputs* y *output_type*) una simple plantilla *jinja2* que puedes refinar.
- El formato de salida esperado.

Puedes mejorar el *prompt* del sistema, reemplazando el *prompt* completo pasando tu propio *prompt* como argumento al parámetro *system_prompt*

In [23]:
from transformers import ReactJsonAgent
from transformers.agents import PythonInterpreterTool

agent = ReactJsonAgent(tools=[PythonInterpreterTool()], system_prompt="{your_custom_prompt}")

Failed to load tokenizer for model meta-llama/Meta-Llama-3.1-8B-Instruct: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-677b6782-0fe0f1431d2d85407e3535ad;71da023d-90c8-4fab-ac99-ec71925c99fb)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct to ask for access.. Loading default tokenizer instead.


### Inspeccionando la ejecución de un agente

Para inspeccionar lo que ocurrió durante una ejecuión, podemos inspeccionar los siguientes atributos:

- *agents.logs* almancena los logs de grano fino del agente. A cada caso de la ejecución del agente, se almancena todo en un diccionario que es añadido a *agents.logs*.
- La ejecución del método *agent.writen_inner_memory_from_logs()* crea una memoria interior de los logs del agente para que se visualizado por el modelo de lenguaje, como una lista de mensajes de chat. Este método sigue cada paso del log y solo almancena como un mensaje aquello en lo que está interesado: por ejemplo, guardará el *prompt* del sistema y la tarea en mensajes separados, luego, para cada paso, almacenará la salida del modelo de lenguaje como mensaje, y la salida de la herramienta invocada como otro mensaje. Usa este método si quieres un vista de nivel más alto de lo qu e suscedió, pero no almacenará cada log.

### Herramientas (*Tools*)

Un herramienta es una función atómica que puede ser utilizada por un agente.

Por ejemplo, la herramienta *PythonInterpreterTool* tiene: un nombre, una descripción, una descripción de la entradas, un tipo de salida y un método *__call__* para realizar la acción.

Cuando se inicializa el agente, los atributos de la herramienta se usan para generar una descripción de la misma que es integrada en el *prompt* del sistema del agente. Esto le muestra al agente qué herramientas utilizar y por qué.

### Caja de herramientas por defecto

La librería *transformers* viene con un caja de herramientas por defecto, que puedes añadir a tu agente durante su inicialización usando el parámetro *add_base_tools=True*:

- *Document question answering*: dado un documento, responde una pregunta en base a su contenido.
- *Image question answering*: dada una imagen, responde una pregunta en base a su contenido.
- *Speech to text*: dado un audio de una persona hablando, transcribe su con discurso a texto.
- *Text to speech*: convierte texto a habla.
- *Translation*: convierte una tarea de un idioma a otro.
- *DuckDuckGo search*: realiza una búsqueda web utilizando el navegador DuckDuckGo.
- *Python code interpreter*: ejecuta código generado por un LLM usando un entorno Python de ejecución segura.

De forma manual podemos usar una herramienta concreta llamando a la función *load_tool()* indicando la tarea a realizar.

In [24]:
from transformers import load_tool

tool = load_tool("text_to_speech")
audio = tool("This is a text to speech tool")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Process Process-auto_conversion:
Process Process-auto_conversion:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/cesga/2020/software/Core/python/3.9.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/cesga/2020/software/Core/python/3.9.9/lib/python3.9/multiprocessing/pr

KeyboardInterrupt: 

### Creación de una nueva herramienta

También puedes crear tu propia herramienta para casos no cubiertos por las herramientas por defecto de HuggingFace. Por ejemplo, creemos una herramienta que identifica y descarga el modelo más descargado de HunggingFace para una tarea determinada. 

Empecemos con el siguiente código:

In [8]:
from huggingface_hub import list_models

task = "text-classification"

model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
print(model.id)

distilbert/distilbert-base-uncased-finetuned-sst-2-english


Este código puede ser convertido de forma sencilla en una herramienta, simplemente envolviéndolo en una herramienta y añadiendo el decorador *tool*.

In [9]:
from transformers import tool

@tool
def model_download_tool(task: str) -> str:
    """
    This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
    It returns the name of the checkpoint.

    Args:
        task: The task for which
    """
    model = next(iter(list_models(filter="text-classification", sort="downloads", direction=-1)))
    return model.id

Esta función necesita:
- Un nombre claro que describa claramente lo que hace la herramienta (*model_download_tool* en este caso).
- Pistas (como comentario) de las entradas y salidas.
- Un descripción, que incluye una parte 'Args:' donde se describe cada argumento (sin indicar su tipo). Todo esto será automáticamente cocinda en el *prompt* de sistema durante la inicialización, por lo tanto asegúrate de hacerlo todo lo claro posible.

Después ya puede inicializar un agente incluyendo esta herramienta y utilizarlo.

In [12]:
from transformers import CodeAgent
from transformers import HfApiEngine

llm_engine = HfApiEngine(model="microsoft/Phi-3.5-mini-instruct")
agent = CodeAgent(tools=[model_download_tool], llm_engine=llm_engine)
agent.run(
    "Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?"
)

======== New task ========
Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?
=== Agent thoughts:

>>> Agent is executing the code below:
most_downloaded_model = model_download_tool({'task': {'type': 'text-to-video'}})
final_answer(most_downloaded_model)
====


'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

### Manejo de la caja de herramientas del agente

Si ya has inicializado un agente, no es cómodo volverlo a inicializar desde cero para incluir una herramienta más en su caja de herramientas. En *Transformers* existen un API para añadir o reemplazar herramientas de la caja de herramientas de un agente.

Añadamos *model_download_tool* a un agente previamente inicializado solo con la caja de herramientas por defecto. Luego, hagamos una prueba de uso del agente.

In [13]:
from transformers import CodeAgent

agent = CodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)
agent.toolbox.add_tool(model_download_tool)
agent.run(
    "Can you read out loud the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub and return the audio?"
)

======== New task ========
Can you read out loud the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub and return the audio?
=== Agent thoughts:
Thought: I will use the tool `model_download_tool` to find the most downloaded model for the 'text-to-video' task on the Hugging Face Hub. Then, I will use `text_to_speech` to read out the model name.
>>> Agent is executing the code below:
most_downloaded_model = model_download_tool(task="text-to-video")
print(f"The most downloaded model is {most_downloaded_model}.")
audio = text_to_speech(text=most_downloaded_model)
final_answer(audio)
====
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current p

Para reemplazar una herramienta podemos usar la función *agent.toolbox.update_tool()*. Esto es útil si la nueva herramienta es un reemplazo uno-a-uno de la herramienta anterior. Asegúrate de que la nueva herramienta tiene la misma API que la herramienta anterior, sinó tendrás que adaptar el *prompt* del sistema para asegurarte de que todos los ejemplos que involucran a la nueva herramienta son adaptados.

### Usa una colección de herramientas

Finalmente podemos construir colecciones de herramientas usando el objeto *ToolCollection*, con el *slug* de la colección que quieres utilizar. Luego, pásale una lista de las herramientas que quieres utilizar a tu agente y empieza a utilizarlas.

In [17]:
from transformers import ToolCollection, ReactCodeAgent
from huggingface_hub import login

login("<your_token>")

image_tool_collection = ToolCollection(collection_slug="huggingface-tools/diffusion-tools-6630bb19a942c2306a2cdb6f")
agent = ReactCodeAgent(tools=[*image_tool_collection.tools], add_base_tools=True)
agent.run("Please draw me a picture of rivers and lakes.")

HfHubHTTPError: (Request ID: Root=1-677b674e-5a9b86932cbe086a597c8bd7;2df626b9-0e1f-4c91-b4ed-60e547dc554f)

403 Forbidden: You don't have the required permissions to complete this action.
Cannot access content at: https://huggingface.co/api/collections/huggingface-tools/diffusion-tools-6630bb19a942c2306a2cdb6f.
Make sure your token has the correct permissions.